In [1]:
!nvidia-smi

Sun Dec 15 17:52:23 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8             22W /   80W |      12MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
import torch.nn as nn

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.current_device()

0

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

In [6]:
from ultralytics import YOLO

In [7]:
# # class ChannelAttention(nn.Module):
# #     def __init__(self, in_channels, reduction_ratio=16):
# #         super(ChannelAttention, self)._init_()
# #         self.avg_pool = nn.AdaptiveAvgPool2d(1)
# #         self.max_pool = nn.AdaptiveMaxPool2d(1)
        
# #         self.fc = nn.Sequential(
# #             nn.Conv2d(in_channels, in_channels // reduction_ratio, 1, bias=False),
# #             nn.ReLU(inplace=True),
# #             nn.Conv2d(in_channels // reduction_ratio, in_channels, 1, bias=False)
# #         )
# #         self.sigmoid = nn.Sigmoid()

# import math

# class ChannelAttention(nn.Module):
#     def __init__(self, in_channels, reduction_ratio=16):
#         super(ChannelAttention, self).__init__()
#         reduced_channels = max(1, in_channels // reduction_ratio)  # Ensure at least 1 channel
#         self.avg_pool = nn.AdaptiveAvgPool2d(1)
#         self.max_pool = nn.AdaptiveMaxPool2d(1)
        
#         self.fc = nn.Sequential(
#             nn.Conv2d(in_channels, reduced_channels, 1, bias=False),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(reduced_channels, in_channels, 1, bias=False)
#         )
#         self.sigmoid = nn.Sigmoid()


#     def forward(self, x):
#         avg_out = self.fc(self.avg_pool(x))
#         max_out = self.fc(self.max_pool(x))
#         out = avg_out + max_out
#         return self.sigmoid(out) * x

# class SpatialAttention(nn.Module):
#     def __init__(self, kernel_size=7):
#         super(SpatialAttention, self)._init_()
#         assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
#         padding = 3 if kernel_size == 7 else 1
        
#         self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         avg_out = torch.mean(x, dim=1, keepdim=True)
#         max_out, _ = torch.max(x, dim=1, keepdim=True)
#         x_concat = torch.cat([avg_out, max_out], dim=1)
#         spatial_att = self.conv(x_concat)
#         return self.sigmoid(spatial_att) * x

# class CBAM(nn.Module):
#     def __init__(self, in_channels, reduction_ratio=16, kernel_size=7):
#         super(CBAM, self)._init_()
#         self.channel_attention = ChannelAttention(in_channels, reduction_ratio)
#         self.spatial_attention = SpatialAttention(kernel_size)

#     def forward(self, x):
#         x = self.channel_attention(x)
#         x = self.spatial_attention(x)
#         return x

# # def modify_yolo_backbone(model):
# #     """
# #     Modify YOLO model to integrate CBAM attention modules
    
# #     Args:
# #         model (ultralytics.nn.tasks.YOLOModel): Original YOLO model
    
# #     Returns:
# #         Modified YOLO model with CBAM attention
# #     """
# #     # Iterate through backbone layers
# #     for name, module in model.model.named_children():
# #         if isinstance(module, nn.Sequential):
# #             for block_name, block in module.named_children():
# #                 # Example: Add CBAM after specific convolutional layers
# #                 if isinstance(block, nn.Conv2d):
# #                     # Insert CBAM module after the convolution
# #                     cbam = CBAM(block.out_channels)
# #                     module.add_module(f'cbam_{block_name}', cbam)
# #                     print("Module added")
# #                 else:
# #                     print("NOOOOOOOO")
# #     return model

# def modify_yolo_backbone(model):
#     """
#     Modify YOLO model to integrate CBAM attention modules
    
#     Args:
#         model (ultralytics.nn.tasks.YOLOModel): Original YOLO model
    
#     Returns:
#         Modified YOLO model with CBAM attention
#     """
#     for name, module in model.model.named_modules():
#         if isinstance(module, nn.Conv2d):
#             # Replace the Conv2d with a new sequential block with CBAM
#             cbam = CBAM(module.out_channels)
#             # Wrap the Conv2d and CBAM into an nn.Sequential
#             new_block = nn.Sequential(
#                 module,
#                 cbam
#             )
#             # Update the parent module to use the new block
#             parent_name = ".".join(name.split(".")[:-1])
#             if parent_name == "":
#                 setattr(model.model, name, new_block)
#             else:
#                 parent_module = dict(model.model.named_modules())[parent_name]
#                 setattr(parent_module, name.split(".")[-1], new_block)
#             print(f"Replaced {name} with Conv2d + CBAM block")
#     return model


import torch
import torch.nn as nn
import math
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super(ChannelAttention, self).__init__()
        reduced_channels = max(1, in_channels // reduction_ratio)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, reduced_channels, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(reduced_channels, in_channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out) * x

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_concat = torch.cat([avg_out, max_out], dim=1)
        spatial_att = self.conv(x_concat)
        return self.sigmoid(spatial_att) * x

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction_ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x

def modify_yolo_backbone(model):
    for name, module in model.model.named_modules():
        if isinstance(module, nn.Conv2d):
            cbam = CBAM(module.out_channels)
            new_block = nn.Sequential(module, cbam)
            parent_name = ".".join(name.split(".")[:-1])
            if parent_name == "":
                setattr(model.model, name, new_block)
            else:
                parent_module = dict(model.model.named_modules())[parent_name]
                setattr(parent_module, name.split(".")[-1], new_block)
            print(f"Replaced {name} with CBAM block")
    return model


In [8]:
model = YOLO("yolo11m.pt")

In [9]:
enhanced_model = modify_yolo_backbone(model)

Replaced model.0.conv with CBAM block
Replaced model.1.conv with CBAM block
Replaced model.2.cv1.conv with CBAM block
Replaced model.2.cv2.conv with CBAM block
Replaced model.2.m.0.cv1.conv with CBAM block
Replaced model.2.m.0.cv2.conv with CBAM block
Replaced model.2.m.0.cv3.conv with CBAM block
Replaced model.2.m.0.m.0.cv1.conv with CBAM block
Replaced model.2.m.0.m.0.cv2.conv with CBAM block
Replaced model.2.m.0.m.1.cv1.conv with CBAM block
Replaced model.2.m.0.m.1.cv2.conv with CBAM block
Replaced model.3.conv with CBAM block
Replaced model.4.cv1.conv with CBAM block
Replaced model.4.cv2.conv with CBAM block
Replaced model.4.m.0.cv1.conv with CBAM block
Replaced model.4.m.0.cv2.conv with CBAM block
Replaced model.4.m.0.cv3.conv with CBAM block
Replaced model.4.m.0.m.0.cv1.conv with CBAM block
Replaced model.4.m.0.m.0.cv2.conv with CBAM block
Replaced model.4.m.0.m.1.cv1.conv with CBAM block
Replaced model.4.m.0.m.1.cv2.conv with CBAM block
Replaced model.5.conv with CBAM block
Repl

In [10]:
print(enhanced_model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): CBAM(
            (channel_attention): ChannelAttention(
              (avg_pool): AdaptiveAvgPool2d(output_size=1)
              (max_pool): AdaptiveMaxPool2d(output_size=1)
              (fc): Sequential(
                (0): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): ReLU(inplace=True)
                (2): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              )
              (sigmoid): Sigmoid()
            )
            (spatial_attention): SpatialAttention(
              (conv): Conv2d(2, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
              (sigmoid): Sigmoid()
            )
          )
        )
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)

In [11]:
enhanced_model.train(data = "/home/sandeep/Vehicle-Defect-Detection/dataset/processed/standard/dataset.yaml", imgsz=640, batch=8, epochs=100, workers=1, device=0)

Ultralytics 8.3.18 🚀 Python-3.12.4 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5931MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/home/sandeep/Vehicle-Defect-Detection/dataset/processed/standard/dataset.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=1, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save

train: Scanning /home/sandeep/Vehicle-Defect-Detection/dataset/processed/standar
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.22 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/sandeep/Vehicle-Defect-Detection/dataset/processed/standard/


Plotting labels to runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000588, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to runs/detect/train6
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        720       2435      0.651      0.563      0.606      0.298



100 epochs completed in 5.188 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 40.5MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.18 🚀 Python-3.12.4 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5931MiB)
YOLO11m summary (fused): 303 layers, 20,040,055 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        720       2435      0.654      0.563      0.606      0.298
                  Burn         33         85      0.521      0.423      0.402      0.191
           Clear Glass         63        107      0.704      0.514      0.586      0.246
             Corrosion        138        461      0.651      0.514      0.573      0.223
                 Crack         95        225      0.657      0.511      0.601      0.249
               Cutouts         13         19      0.741      0.737      0.768      0.398
         Defaced Paint        119        376      0.634      0.636      0.637      0.291
                 Dents         93        152      0.689      0.605      0.668      0.384
        Hairline Glass         62         84      0.359      0.131      0.193     0.0788
             Inclusion        145        434      0.637      0.484      0.563      0.279
            Paint-Peel        168        324       0.75       0.73      0.772       0.36
         Spidery Glas

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78d12b4065d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

___

SyntaxError: invalid syntax (3631438787.py, line 1)